# Problem Simulation Tutorial

In [20]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.2'

Before configuring and solving a problem with real data, it may be a good idea to perform Monte Carlo analysis on simulated data to verify that it is possible to accurately estimate model parameters. For example, before configuring and solving the example problems in the prior tutorials, it may have been a good idea to simulate data according to the assumed models of supply and demand. During such Monte Carlo anaysis, the data would only be used to determine sample sizes and perhaps to choose reasonable true parameters.

Simulations are configured with the :class:`Simulation` class, which requires many of the same inputs as :class:`Problem`. The two main differences are:

1. Variables in formulations that cannot be loaded from `product_data` or `agent_data` will be drawn from independent uniform distributions.
2. True parameters and the distribution of product unobservables are specified.

First, we'll use :func:`build_id_data` to build market and firm IDs for a model in which there are $T = 50$ markets, and in each market $t$, a total of $J_t = 20$ products produced by $F = 10$ firms.

In [21]:
id_data = pyblp.build_id_data(T=600, J=4, F=4)
id_data

rec.array([([0], [0]), ([0], [1]), ([0], [2]), ..., ([599], [1]),
           ([599], [2]), ([599], [3])],
          dtype=[('market_ids', 'O', (1,)), ('firm_ids', 'O', (1,))])

Next, we'll create an :class:`Integration` configuration to build agent data according to a Gauss-Hermite product rule that exactly integrates polynomials of degree $2 \times 9 - 1 = 17$ or less.

In [22]:
integration = pyblp.Integration('product', 9)
integration

Configured to construct nodes and weights according to the level-9 Gauss-Hermite product rule with options {}.

We'll then pass these data to :class:`Simulation`. We'll use :class:`Formulation` configurations to create an $X_1$ that consists of a constant, prices, and an exogenous characteristic; an $X_2$ that consists only of the same exogenous characteristic; and an $X_3$ that consists of the common exogenous characteristic and a cost-shifter.

In [23]:
simulation = pyblp.Simulation(
   product_formulations=(
       pyblp.Formulation('0 + prices + x + satellite + wired'),
       pyblp.Formulation('0 + satellite + wired'),
       pyblp.Formulation('1 + w')
   ),
   beta=[-2, 1, 4, 4],
   sigma=np.eye(2),
   gamma=[1/2, 1/4],
   product_data=id_data,
   integration=integration,
   seed=1995
)
simulation

Dimensions:
 T    N     F     I     K1    K2    K3 
---  ----  ---  -----  ----  ----  ----
600  2400   4   48600   4     2     2  

Formulations:
        Column Indices:              0        1        2        3  
-------------------------------  ---------  -----  ---------  -----
  X1: Linear Characteristics      prices      x    satellite  wired
 X2: Nonlinear Characteristics   satellite  wired                  
X3: Linear Cost Characteristics      1        w                    

Nonlinear Coefficient True Values:
 Sigma:    satellite   wired  
---------  ---------  --------
satellite  +1.0E+00           
  wired    +0.0E+00   +1.0E+00

Beta True Values:
 prices      x      satellite   wired  
--------  --------  ---------  --------
-2.0E+00  +1.0E+00  +4.0E+00   +4.0E+00

Gamma True Values:
   1         w    
--------  --------
+5.0E-01  +2.5E-01

When :class:`Simulation` is initialized, it constructs :attr:`Simulation.agent_data` and simulates :attr:`Simulation.product_data`.

The :class:`Simulation` can be further configured with other arguments that determine how product unobservables are simulated and how marginal costs are specified.

At this stage, simulated variables are not consistent with true parameters, so we still need to solve the simulation with :meth:`Simulation.replace_endogenous`. This method replaced simulated prices and market shares with values that are consistent with the true parameters. Just like :meth:`ProblemResults.compute_prices`, to do so it iterates over the $\zeta$-markup equation from :ref:`references:Morrow and Skerlos (2011)`.

In [24]:
simulation_results = simulation.replace_endogenous()
simulation_results

Simulation Results Summary:
Computation  Fixed Point  Fixed Point  Contraction  Profit Gradients  Profit Hessians  Profit Hessians
   Time       Failures    Iterations   Evaluations      Max Norm      Min Eigenvalue   Max Eigenvalue 
-----------  -----------  -----------  -----------  ----------------  ---------------  ---------------
 00:00:01         0          15713        15713         +4.5E-13         -1.5E+00         -1.2E-03    

Now, we can try to recover the true parameters by creating and solving a :class:`Problem`. 

The convenience method :meth:`SimulationResults.to_problem` constructs some basic "sums of characteristics" BLP instruments that are functions of all exogenous numerical variables in the problem. In this example, excluded demand-side instruments are the cost-shifter `z` and traditional BLP instruments constructed from `x`. Excluded supply-side instruments are traditional BLP instruments constructed from `x` and `z`.

In [25]:
problem = simulation_results.to_problem()

In [26]:
problem

Dimensions:
 T    N     F     I     K1    K2    K3    MD    MS 
---  ----  ---  -----  ----  ----  ----  ----  ----
600  2400   4   48600   4     2     2     7     6  

Formulations:
        Column Indices:              0        1        2        3  
-------------------------------  ---------  -----  ---------  -----
  X1: Linear Characteristics      prices      x    satellite  wired
 X2: Nonlinear Characteristics   satellite  wired                  
X3: Linear Cost Characteristics      1        w                    

In [27]:
problem.products.ZD

array([[1.33027006, 1.20997016, 1.40970248, ..., 0.75298194, 0.61986679,
        0.90327761],
       [1.92380124, 1.98061237, 1.8859878 , ..., 0.27669662, 0.02633561,
        0.13263541],
       [1.50223858, 1.14521282, 1.37692797, ..., 0.78575645, 0.44789827,
        0.96803496],
       ...,
       [1.32571638, 2.17032115, 2.37319543, ..., 0.87353479, 0.21796982,
        0.15147787],
       [1.19834391, 1.71110727, 2.27920326, ..., 0.96752697, 0.3453423 ,
        0.61069175],
       [1.43161993, 1.51121454, 2.71221748, ..., 0.53451274, 0.11206628,
        0.81058448]], shape=(2400, 7))

In [37]:
problem.products.ZS

array([[0.61530516, 0.61986679, 0.90327761, 0.75298194, 1.        ,
        0.57585458],
       [1.16439555, 0.02633561, 0.13263541, 0.27669662, 1.        ,
        0.02676419],
       [1.00067827, 0.44789827, 0.96803496, 0.78575645, 1.        ,
        0.19048147],
       ...,
       [1.23537468, 0.21796982, 0.15147787, 0.87353479, 1.        ,
        0.13528654],
       [0.5384475 , 0.3453423 , 0.61069175, 0.96752697, 1.        ,
        0.83221372],
       [1.25052367, 0.11206628, 0.81058448, 0.53451274, 1.        ,
        0.12013755]], shape=(2400, 6))

We'll choose starting values that are half the true parameters so that the optimization routine has to do some work. Note that since we're jointly estimating the supply side, we need to provide an initial value for the linear coefficient on prices because this parameter cannot be concentrated out of the problem (unlike linear coefficients on exogenous characteristics).

In [38]:
results = problem.solve(
    sigma=0.5 * simulation.sigma, 
    beta=[0.5 * simulation.beta[0, 0], None, None, None],
    optimization=pyblp.Optimization('l-bfgs-b', {'gtol': 1e-5})
)
results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.4E+01     +1.3E-05        +2.1E+01         +8.5E+02         0         +1.2E+03          +4.0E+03     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:29       Yes          19           26         160757       500504   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
 Sigma:    satellite     wired   
---------  ----------  ----------
satellite   +8.3E-01             
           (+2.7E-01)            
     

The parameters seem to have been estimated reasonably well.

In [39]:
np.c_[simulation.beta, results.beta]

array([[-2.        , -1.93527134],
       [ 1.        ,  0.9462575 ],
       [ 4.        ,  3.90220011],
       [ 4.        ,  3.82957091]])

In [40]:
np.c_[simulation.gamma, results.gamma]

array([[0.5       , 0.4704242 ],
       [0.25      , 0.26426238]])

In [41]:
np.c_[simulation.sigma, results.sigma]

array([[1.        , 0.        , 0.82865619, 0.        ],
       [0.        , 1.        , 0.        , 0.8427668 ]])

In addition to checking that the configuration for a model based on actual data makes sense, the :class:`Simulation` class can also be a helpful tool for better understanding under what general conditions BLP models can be accurately estimated. Simulations are also used extensively in pyblp's test suite.